In [0]:
df = spark.read.load("/Volumes/luffy/phase2/gold/gold_predictions_raw")
display(df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,prediction,probability
14586,340.55999999999995,27,652,2011-04-26,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"
14981,102.11999999999998,8,97,2011-04-07,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"
18184,49.8,1,60,2011-11-24,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"
14765,254.10999999999996,39,135,2011-11-10,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"
12607,3159.019999999999,202,2456,2011-10-12,1,1.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}"
14223,1007.0300000000001,61,587,2011-09-08,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"
16766,1145.6,34,536,2011-03-11,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"
15110,1021.95,40,252,2011-11-16,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"
12406,3549.8500000000013,108,1827,2011-11-17,1,1.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}"
16361,896.6600000000001,106,421,2011-11-30,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}"


In [0]:
from pyspark.sql.functions import udf,col
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import DoubleType

def get_high_value_prob(prob_vector):
    return float(prob_vector[1])

get_high_value_prob_udf = udf(
    get_high_value_prob,
    DoubleType()
)

scored_df = df.withColumn(
    "high_value_prob",
    get_high_value_prob_udf(col("probability"))
)

display(scored_df.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,is_high_valued,prediction,probability,high_value_prob
14586,340.55999999999995,27,652,2011-04-26,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0
14981,102.11999999999998,8,97,2011-04-07,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0
18184,49.8,1,60,2011-11-24,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0
14765,254.10999999999996,39,135,2011-11-10,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0
12607,3159.019999999999,202,2456,2011-10-12,1,1.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}",1.0
14223,1007.0300000000001,61,587,2011-09-08,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0
16766,1145.6,34,536,2011-03-11,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0
15110,1021.95,40,252,2011-11-16,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0
12406,3549.8500000000013,108,1827,2011-11-17,1,1.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""0.0"",""1.0""]}",1.0
16361,896.6600000000001,106,421,2011-11-30,0,0.0,"{""type"":""1"",""size"":null,""indices"":null,""values"":[""1.0"",""0.0""]}",0.0


In [0]:
top_buyers = scored_df.filter(
    (col("prediction") == 1) &
    (col("high_value_prob") > 0.65)
)
top_buyers = top_buyers.select("CustomerID","total_spent","total_transactions","total_quantity","last_purchase_date","high_value_prob").orderBy(
    col("high_value_prob").desc()
)

In [0]:
top_buyers.count(),len(top_buyers.columns)

(891, 6)

In [0]:
spark.read.load("/Volumes/luffy/phase2/silver/data_for_training_or_prediction/").count()

4373

## Out of 4373 buyers, only 891 are top buyers

In [0]:
display(top_buyers.head(10))

CustomerID,total_spent,total_transactions,total_quantity,last_purchase_date,high_value_prob
17449,2536.8599999999997,279,1129,2011-12-07,1.0
13901,2867.2,149,1739,2011-09-28,1.0
18272,3092.38,170,2056,2011-12-07,1.0
12406,3549.8500000000013,108,1827,2011-11-17,1.0
12517,2540.789999999999,286,1394,2011-11-11,1.0
12359,6499.630000000005,254,1632,2011-12-02,1.0
14132,3591.930000000001,201,3370,2011-12-07,1.0
13488,9026.01,427,4524,2011-11-22,1.0
16928,2913.699999999998,213,2134,2011-09-20,1.0
13050,5989.109999999998,422,3789,2011-11-30,1.0


In [0]:
top_buyers.write.format("delta").mode("overwrite").save("/Volumes/luffy/phase2/gold/top_buyers")